# Métodos de Compresión - Breast Cancer Dataset

In [ ]:
######################################################################
#	MODEL COMPRESSION TECHNIQUES EXAMPLE. 2022.
######################################################################
# Copyright (C) 2022. J.D.Diaz-Delgado (JDD) jd.diazd@uniandes.edu.co
# 
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, version 3 of the License.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>
####################################################################*/

## Introducción a las Técnicas de Compresión

Las técnicas de compresión son utilizadas para reducir el tamaño de modelos de machine learning con el objetivo de mejorar eficiencia, evitar la sobre-parametrización y optimizar el uso de recursos. Existen diferentes técnicas de compresión, en este caso se usa la poda de parámetros, Knowledge Distillation y la Cuantificación. A continuación se construirá el modelo base y posteriormente se aplicarán estas técnicas para reducir su tamaño.

El modelo que se utiliza en este ejemplo se encarga de realizar una clasificación binaria entre benigno y maligno de la biopsia de una masa en el pecho de un paciente. Se usan 30 características con el fin de determinar si es cancerigeno o no. Este dataset de 569 instancias, con el que se entrenarán y validarán los modelos, le pertenece a la Universidad de Wisconsin y se puede encontrar [aquí](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic))<br>

## Construcción del Modelo Base
En esta sección nos dedicaremos a hacer el preprocesamiento de los datos y a generar el primer modelo del cual partiremos para usar las técnicas de compresión. Empezaremos importando las librerias que usaremos en este ejemplo. La construcción de este modelo base hace parte de un tutorial realizado por Dr. Sreenivas Bhattiprolu que se puede encontrar [aquí](https://colab.research.google.com/drive/1WEZxybgoxQz8Lmp_r6Zq6OHYdvwaz2Df?usp=sharing)<br>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tempfile
import tensorflow as tf

import tensorflow_model_optimization as tfmot

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from sklearn.metrics import confusion_matrix

Se tiene un archivo:

`data.csv`: posee las 30 características de los núcleos de las células tomadas de una biopsia por aspiración con aguja fina sobre cada usuario, identificado con un ID único. Son 10 características principales de las cuales se toma la media (*_mean*), el error estándar (*_se*) y el peor resultado encontrado en la biopsia (*_worst*). Puede observar los nombres de las 10 características principales a continuación:

- ID: número de paciente
- DIAGNOSIS: resultado del diagnóstico; Benigno (B) o Maligno (M)

- RADIUS: distancia desde el centro a puntos en el perímetro
- TEXTURE: textura
- PERIMETER: perimetro
- AREA: área
- SMOOTHNESS: variación local en tamaños del radio
- COMPACTNESS: perimetro^2 / área - 1.0
- CONCAVITY: severidad de las porciones cóncavas del contorno
- CONCAVE_POINTS: número de porciones cóncavas del contorno
- SYMMETRY: simetría
- FRACTAL_DIMENSION: dimensión fractal (aproximación de línea de costa - 1)

A continuación se muestran las primeras instancias del dataset.

In [ ]:
df = pd.read_csv("data.csv")
df.head()

Vemos que la columna 32 no tiene ninguna característica asignada y ninguno de los datos de esta columna aporta valor. Por esta razón podemos retirarla y volvemos a imprimir las primeras filas para verificar el cambio.

In [ ]:
df.drop('Unnamed: 32', axis=1, inplace=True)

In [ ]:
df.head()

Una buena práctica es verificar si hay valores faltantes por cada característica para tomar la decisión de retirar una característica por una gran falta de valores o retirar filas específicas que no cuenten con todos los valores. Podemos verificar esto usando `df.isnull().sum()`

In [ ]:
print(df.isnull().sum())

En este caso no es necesario eliminar ninguna columna debido a que todas contienen datos. Pasamos ahora a renombrar la columna de `diagnosis` a `label` para tener claro que esta columna contiene el resultado esperado de la clasificación. Adicionalmente, graficaremos la distribución de ambas clases para determinar si hay un desbalance significativo de los datos.

In [ ]:
df = df.rename(columns = {'diagnosis' : 'label'})
print(df.dtypes)

In [ ]:
count_plots = sns.countplot(x = "label", data = df)
print("Distribución de los datos: ", df['label'].value_counts())

En la gráfica podemos observar que existe un desbalance entre clases cercano a la razón 4:6. Sin embargo, es un desbalance leve por lo que no es necesario darle un tratamiento diferente. Continuamos con la codificación de etiquetas para hacer el cambio de 'B' y 'M' a 0 y 1, respectivamente. Esto para poder dar un valor a las clases y entrenar el modelo. Para la codificación usamos `LabelEncoder()`. Luego de completada la codificación y de definir `y` como la columna `label`, podemos retirar esta columna y la de `id` (que no aporta nada al entrenamiento) para preparar los datos para el entrenamiento.

In [ ]:
y = df['label'].values
print("Las etiquetas antes de la codificación son: ", np.unique(y))

le = LabelEncoder()
Y = le.fit_transform(y)
print("Las etiquetas luego de la codificación son: ", np.unique(Y))

In [ ]:
X = df.drop(labels = ['label', 'id'], axis = 1)
X.head()

En la tabla que acabamos de imprimir vemos que las características toman valores en rangos muy variados: radio entre 17 y 20, perimetro entre 77 y 135, area entre 386 y 1326, etc. Esto hace que sea más dificil el proceso de entrenamiento, por esta razón usaremos la función `MinMaxScaler()` que se encarga de escalar cada característica de forma individual a un rango entre 0 y 1 con el objetivo de obtener un modelo con mejor rendimiento.

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)
print(X)

Ahora haremos el split de datos de entrenamiento y de validación. Tomaremos el 25% del dataset para validación y el resto para entrenamiento.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)
print("La forma de los datos de entrenamiento es: ", X_train.shape)
print("La forma de los datos de validación es: ", X_test.shape)

Ya con la partición de los datos en entrenamiento y validación, podemos crear nuestro modelo de red neuronal. Para este ejemplo usaremos Keras y la armaremos de forma secuencial, lo que significa que añadiremos capa por capa. Tendremos una red neuronal totalmente conectada con 30 neuronas de entrada (para las 30 características), luego una capa oculta de 16 neuronas y por último una neurona de salida. Adicionalmente usamos una capa de `Dropout(0.2)`, entre la capa oculta y la de salida, para que de forma aleatoria el 20% de los pesos se vayan a cero para prevenir un sobreajuste a los datos. Como función de activación se usará una sigmoide. 

Imprimimos el resumen del modelo en el que se muestra cada una de las capas y la cantidad de parámetros.

In [ ]:
model = Sequential()
model.add(Dense(16, input_dim = 30, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print(model.summary())

Luego de haber terminado con el preprocesamiento y de haber escogido una arquitectura para nuestra red neuronal podemos empezar el entrenamiento!! Este proceso lo guardaremos en la variable `history_base` para luego poder analizarlo. Entrenaremos la red por 100 epochs usando un batch size de 64. Si hay problemas de memoria, pueden disminuír el batch size en potenicas de 2.

In [ ]:
history_base = model.fit(X_train, y_train, verbose = 1, epochs = 100, batch_size = 64, validation_data = (X_test, y_test))

Finalizado el entrenamiento, podemos analizar el comportamiento de las pérdidas y la precisión a medida que avanzamos de epoch. A continuación podremos ver 2 gráficas relacionadas a estos indicadores.

In [ ]:
loss = history_base.history['loss']
val_loss = history_base.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label='Pérdida de entrenamiento')
plt.plot(epochs, val_loss, 'b', label='Pérdida de validación')
plt.title('Pérdida de entrenamiento y validación')
plt.xlabel('Epochs')
plt.ylabel('Pérdida')
plt.legend()
plt.savefig('perdida_modelo_base.png')
plt.show()

acc = history_base.history['accuracy']  # Use accuracy si acc no funciona
val_acc = history_base.history['val_accuracy']  # Use val_accuracy si acc no funciona
plt.plot(epochs, acc, 'g', label='Precisión de entrenamiento')
plt.plot(epochs, val_acc, 'b', label='Precisión de validación')
plt.title('Precisión de entrenamiento y validación')
plt.xlabel('Epochs')
plt.ylabel('Precisión')
plt.legend()
plt.savefig('precision_modelo_base.png')
plt.show()

Adicionalmente, podemos graficar la matríz de confusión que nos muestra que tan bueno es el modelo prediciendo para ambas clases.

In [ ]:
# Predecir los resultados del bloque de validación
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

# Hacer la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

matriz_modelo_base = sns.heatmap(cm, annot=True, cmap="crest")
fig = matriz_modelo_base.get_figure()
fig.savefig("matriz_modelo_base.png")

## Técnica de Compresión - Poda de parámetros
Ya teniendo nuestro modelo base construido, podemos pasar a aplicar una de las técnicas de compresión para disminuir el tamaño del modelo. Empezaremos con la poda de parámetros. Esta técnica consiste en eliminar de forma iterativa los parámetros con menor peso, es decir, eliminar los parámetros que cuentan con el menor poder predictivo en el modelo. De esta manera, podremos llevar muchos de los pesos a 0 sin que el modelo pierda mucha precisión. En este ejemplo usaremos la librería de `tensorflow_model_optimization`. El uso de esta técnica está basado en el ejemplo de TensorFlow que se puede encontrar [aquí](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras)<br>

Primero haremos un reconteo de los parámetros capa por capa y los imprimiremos para ver sus valores.

In [ ]:
count = 0
for element in model.layers[0].get_weights()[0]:
    for i in element:
        # if i != 0: count += 1
        count += 1
for j in model.layers[0].get_weights()[1]:
    # if j != 0: count +=1
    count += 1
    
for element in model.layers[2].get_weights()[0]:
    for i in element:
        count += 1
        
for j in model.layers[2].get_weights()[1]:
    count += 1

print("Total número de parámetros: ", count)
print()

weights = model.get_weights()
print(weights)

Observamos que tenemos 513 parámetros en total y la gran mayoría de estos pesos es diferente de cero. Ahora guardaremos el modelo base en formato `SavedModel` que es el formato que recomienda TensorFlow en vez de la versión comprimida `.h5`. Para la implementación en FPGA se guardarán los pesos y biases en archivos `.csv`. Es importante eliminar la primera fila y la primera columna de estos archivos para no tener problemas en la implementación.

In [ ]:
_, baseline_model_accuracy = model.evaluate(X_test, y_test, verbose = 0)
print('Precisión modelo base:', baseline_model_accuracy)

# Modelo Base - Formato SavedModel
model.save("modelo_base")

# Archivos de Pesos y Biases para implementación en FPGA
w1_base = model.layers[0].get_weights()[0]
b1_base = model.layers[0].get_weights()[1]
w2_base = model.layers[2].get_weights()[0]
b2_base = model.layers[2].get_weights()[1]

pd.DataFrame(w1_base).to_csv("w1_base.csv")
pd.DataFrame(b1_base).to_csv("b1_base.csv")
pd.DataFrame(w2_base).to_csv("w2_base.csv")
pd.DataFrame(b2_base).to_csv("b2_base.csv")

Adicionalmente, haremos la conversión de este modelo a TensorFlowLite para hacer la implementación en el microcontrolador. No se usará optimización en el proceso de conversión para poder compararlo posteriormente con los modelos cuantificados.

In [ ]:
# Conversión del modelo
converter = tf.lite.TFLiteConverter.from_saved_model("modelo_base")
tflite_model = converter.convert()
open("modelo_base.tflite", "wb").write(tflite_model)

Ahora usaremos parte del ejemplo dado por TensorFlow para crear el modelo que vamos a podar con sus respectivos parámetros de poda. En este caso vamos a entrenar el modelo por la misma cantidad de epochs (100), mismo batch size (64) y mismo porcentaje para bloque de validación (25%) que el modelo base. Adicionalmente, iniciaremos con un sparsity del 30% y finalizaremos con un sparsity del 60%. Esto quiere decir que se removerá luego de la primera iteración el 30% de los parámetros y se seguirán removiendo iterativamente hasta llegar al 60% en el último epoch. Como salida de este bloque de código tenemos un resumen del modelo que usaremos para podar.

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Calcule el paso final para finalizar la poda luego de 100 epochs.
batch_size = 64
epochs = 100
validation_split = 0.25 # 25% de los datos se usará para validación. 

X_num = X_train.shape[0] * (1 - validation_split)
end_step = np.ceil(X_num / batch_size).astype(np.int32) * epochs

# Defina el modelo para podar.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.30,
                                                               final_sparsity=0.60,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requiere una recompilación.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model_for_pruning.summary()

Ahora pasamos a entrenar el modelo para podar!

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir = logdir),
]

history_poda = model_for_pruning.fit(X_train, y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Luego de entrenar este modelo, podemos pasar a analizarlo con las mismas gráficas que mostramos para el modelo base y mostrando la matriz de confusión.

In [ ]:
loss = history_poda.history['loss']
val_loss = history_poda.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label='Pérdida de entrenamiento')
plt.plot(epochs, val_loss, 'b', label='Pérdida de validación')
plt.title('Pérdida de entrenamiento y validación')
plt.xlabel('Epochs')
plt.ylabel('Pérdida')
plt.legend()
plt.savefig('perdida_modelo_podado.png')
plt.show()

acc = history_poda.history['accuracy']  # Use accuracy si acc no funciona
val_acc = history_poda.history['val_accuracy']  # Use val_accuracy si acc no funciona
plt.plot(epochs, acc, 'g', label='Precisión de entrenamiento')
plt.plot(epochs, val_acc, 'b', label='Precisión de validación')
plt.title('Precisión de entrenamiento y validación')
plt.xlabel('Epochs')
plt.ylabel('Precisión')
plt.legend()
plt.savefig('precision_modelo_podado.png')
plt.show()

In [ ]:
# Predecir los resultados del bloque de validación
y_pred_poda = model_for_pruning.predict(X_test)
y_pred_poda = (y_pred_poda > 0.5)

# Hacer la matriz de confusión
cm_poda = confusion_matrix(y_test, y_pred_poda)

matriz_poda = sns.heatmap(cm_poda, annot=True, cmap="crest")
fig = matriz_poda.get_figure()
fig.savefig("matriz_modelo_podado.png")

Observamos que durante el proceso de entrenamiento, cerca al epoch 20, 40, 60 y 80 hay una pérdida en precisión y un aumento en pérdida significativo. Esto se da porque en estos epochs se removió un parámetro con un poder predictivo considerable. Sin embargo, el modelo aprende a predecir con los parámetros que le quedan y recupera su precisión rápidamente. En la matriz de confusión vemos que el comportamiento es muy similar al modelo base, por lo que no hay una pérdida en desempeño significativa.

In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   X_test, y_test, verbose=0)

print('Precisión del modelo base:', baseline_model_accuracy)
print('Precisión del modelo podado:', model_for_pruning_accuracy)

Vemos que al evaluar el modelo con el set de validación, no hay impacto en la precisión respecto al modelo base. Esto nos muestra que la poda de parámetros ha sido muy efectiva y vale la pena hacer uso de este modelo comprimido. A continuación haremos un conteo de los parámetros de la red diferentes a cero para ver cuál fue el impacto de la compresión.

In [ ]:
count = 0
for element in model_for_pruning.layers[0].get_weights()[0]:
    for i in element:
        if i != 0: count += 1
for j in model_for_pruning.layers[0].get_weights()[1]:
    if j != 0: count +=1
    
for element in model_for_pruning.layers[2].get_weights()[0]:
    for i in element:
        if i != 0: count += 1
        
for j in model_for_pruning.layers[2].get_weights()[1]:
    if j != 0: count += 1

print("Total número de parámetros: ", count)
print()

weights = model_for_pruning.get_weights()
print(weights)

Vemos que el número de parámetros diferentes a cero se redujo drásticamente. Esto se puede evidenciar en el conteo de parámetros y visualmente al imprimir los pesos del modelo podado. Ahora guardaremos el modelo base en formato `SavedModel` que es el formato que recomienda TensorFlow en vez de la versión comprimida `.h5`. Para la implementación en FPGA se guardarán los pesos y biases en archivos `.csv`. Es importante eliminar la primera fila y la primera columna de estos archivos para no tener problemas en la implementación.

In [ ]:
# Modelo Podado - Formato SavedModel
model_for_pruning.save("modelo_podado")

# Archivos de Pesos y Biases para implementación en FPGA
w1_podado = model_for_pruning.layers[0].get_weights()[0]
b1_podado = model_for_pruning.layers[0].get_weights()[1]
w2_podado = model_for_pruning.layers[2].get_weights()[0]
b2_podado = model_for_pruning.layers[2].get_weights()[1]

pd.DataFrame(w1_podado).to_csv("w1_podado.csv")
pd.DataFrame(b1_podado).to_csv("b1_podado.csv")
pd.DataFrame(w2_podado).to_csv("w2_podado.csv")
pd.DataFrame(b2_podado).to_csv("b2_podado.csv")

Adicionalmente, haremos la conversión de este modelo a TensorFlowLite para hacer la implementación en el microcontrolador. No se usará optimización en el proceso de conversión para poder compararlo posteriormente con los modelos cuantificados. Debido a que el modelo podado no quedó bien guardado porque contaba con parámetros basura que se incluyeron durante el proceso de poda, se genera a continuación otro modelo con la misma arquitectura y se le cargan los parámetros relevantes para poder hacer la conversión del modelo correctamente. El modelo arreglado quedará guardado en la carpeta `modelo_podado_fix`.

In [ ]:
modelo_podado_fix = keras.Sequential()
modelo_podado_fix.add(keras.layers.Dense(16, input_dim = 30, activation = 'relu', trainable=False))
modelo_podado_fix.add(keras.layers.Dropout(0.2))
modelo_podado_fix.add(keras.layers.Dense(1, trainable=False))
modelo_podado_fix.add(keras.layers.Activation('sigmoid'))

modelo_podado_fix.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

modelo_podado_fix.layers[0].set_weights(model_for_pruning.layers[0].get_weights())
modelo_podado_fix.layers[2].set_weights(model_for_pruning.layers[2].get_weights())

# Modelo Podado Arreglado - Formato SavedModel
modelo_podado_fix.save("modelo_podado_fix")

# Conversión del modelo
converter = tf.lite.TFLiteConverter.from_saved_model("modelo_podado_fix")
tflite_model = converter.convert()
open("modelo_podado.tflite", "wb").write(tflite_model)

# print(modelo_podado_fix.summary())

## Técnicas de Compresión - Knowledge Distillation

Esta técnica de compresión fue propuesta por Google en el 2015 y consiste en entrenar una red neuronal profesora para luego transferirle el conocimiento a una red neuronal estudiante que tiene un menor tamaño. Este método se basa en la idea de que es posible comprimir el conocimiento y reducir drásticamente el tamaño de los modelos sin comprometer la capacidad de predicción. Los resultados obtenidos por Google fueron usando el dataset de MNIST. En este caso, trataremos de comprimir el conocimiento de nuestro modelo base y mantener una buena precisión. Iniciaremos definiendo la clase `Distiller()` que modifica las funciones `train_step`, `test_step` y `compile()`. El tutorial de esta técnica está basado en la implementación de Kenneth Borup. Por lo tanto, el uso de esta clase y de la técnica en general está detallado allí. 

La implementación de Kenneth Borup se puede encontrar [aquí](https://keras.io/examples/vision/knowledge_distillation/)<br>

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

Luego de haber definido la clase `Distiller()`, pasamos a crear los modelos estudiante y profesor. Estos modelos son creados usando `Sequential()`. Inicialmente vamos a definir la red profesora con 16 neuronas en la capa oculta (como en el modelo base) y la red estudiante con 5 neuronas en la capa oculta.

In [ ]:
capa_oculta_profesora = 16 # Cantidad de neuronas en la capa oculta de la red profesora
capa_oculta_estudiante = 5 # Cantidad de neuronas en la capa oculta de la red estudiante

# Crear red profesora
teacher = keras.Sequential(name = "teacher")
teacher.add(keras.layers.Dense(capa_oculta_profesora, input_dim = 30, activation = 'relu'))
teacher.add(keras.layers.Dropout(0.2))
teacher.add(keras.layers.Dense(1))
teacher.add(keras.layers.Activation('sigmoid'))

# Crear red estudiante
student = keras.Sequential(name = "student")
student.add(keras.layers.Dense(capa_oculta_estudiante, input_dim = 30, activation = 'relu'))
student.add(keras.layers.Dropout(0.2))
student.add(keras.layers.Dense(1))
student.add(keras.layers.Activation('sigmoid'))

# Clonar red estudiante para comparación posterior
student_scratch = keras.models.clone_model(student)

Ya definidos los dos modelos, podemos pasar al entrenamiento de la red profesora.

In [ ]:
teacher.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print(teacher.summary())

teacher.fit(X_train, y_train, epochs = 100, batch_size = 64, validation_data = (X_test, y_test))
teacher.evaluate(X_test, y_test)

Ahora pasamos a destilar el conocimiento de la red profeosra a la red estudiante.

In [ ]:
# Initialize and compile distiller

distiller = Distiller(student=student, teacher=teacher)
# distiller.compile(
#     optimizer=keras.optimizers.Adam(),
#     metrics=[keras.metrics.Accuracy()],
#     student_loss_fn=keras.losses.BinaryCrossentropy(),
#     distillation_loss_fn=keras.losses.KLDivergence(),
#     alpha=0.1,
#     temperature=10,
# )

distiller.compile(optimizer = 'adam', metrics = ['accuracy'], student_loss_fn = keras.losses.BinaryCrossentropy(), distillation_loss_fn = keras.losses.BinaryCrossentropy(), alpha = 0.1, temperature = 10)

# Distill teacher to student
distiller.fit(X_train, y_train, epochs=100, batch_size = 64, validation_data = (X_test, y_test))

# Evaluate student on test dataset
distiller.evaluate(X_test, y_test)

# weights = distiller.get_weights()
# print(weights)
# print(student.summary())

# model.save("distilled_model.h5")

Ahora vamos a entrenar la red estudiante por separado para ver qué beneficios trae usar knowledge distillation en este caso.

In [ ]:
student_scratch.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

student_scratch.fit(X_train, y_train, epochs=100, batch_size = 64, validation_data = (X_test, y_test))
student_scratch.evaluate(X_test, y_test)

Luego de aplicar la técnica de compresión de Knowledge Distillation se evidencia que el conocimiento se transmite de forma satisfactoria de la red profesora a la red estudiante perdiendo únicamente un punto de precisión. Sin embargo, al entrenar la red estudiante directamente observamos que se obtiene una precisión mucho más cercana a la de la red profesora. Esto quiere decir que a pesar que la transferencia de conocimiento de una red a otra es posible, pero no es eficiente en este caso. Por lo tanto, descartaremos esta técnica de compresión debido a que no está aportando valor a este modelo.

## Técnicas de Compresión - Cuantización

Por último, vamos a revisar la cuantización de los modelos como técnica de compresión. Esta técnica consiste en cambiar la representación de los pesos, biases y/o entradas con el fin de simplificar las operaciones que se realizan en el proceso de inferencia o para disminuir el tamaño de los modelos al disminuir la cantidad de bits con los que se representan estos números. Normalmente, la cuantización viene acompañada de pérdidas en precisión y es precisamente lo que se va a evaluar en este ejemplo. Acá se hará la conversión a modelos de TFLite con cuantización a Float16 (fp16) y de rango dinámico (drq) y se va a evaluar su desempeño en el microcontrolador.

In [ ]:
# Conversión del modelo - Cuantización con Float16
converter = tf.lite.TFLiteConverter.from_saved_model("modelo_base")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()
open("modelo_cuantizado_fp16.tflite", "wb").write(tflite_model)

# Conversión del modelo - Cuantización con Dynamic Range
converter = tf.lite.TFLiteConverter.from_saved_model("modelo_base")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("modelo_cuantizado_drq.tflite", "wb").write(tflite_model)